# Google Landmark Detection 

## References

- Weyand, T., Araujo, A., Cao, B., & Sim, J. (2020).  
  *Google Landmarks Dataset v2 – A Large-Scale Benchmark for Instance-Level Recognition and Retrieval.*  
  In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR).  

- Tolias, G., Jégou, H., & Vedaldi, A. (2019).  
  *Detect-to-Retrieve: Efficient Regional Aggregation for Image Search.*  
  In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR).  


## 1. Importing Libraries 

In [2]:
# Importing Libs
import os 
import cv2 
import keras 
import random 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from PIL import Image 

## 2. Loading Dataset

In [4]:
df = pd.read_csv("train.csv")

In [5]:
df.head() 

,id,landmark_id
0,17660ef415d37059,1
1,92b6290d571448f6,1
2,cd41bf948edc0340,1
3,fb09f1e98c6d2f70,1
4,25c9dfc7ea69838d,7


In [7]:
df.tail()

,id,landmark_id
1580465,72c3b1c367e3d559,203092
1580466,7a6a2d9ea92684a6,203092
1580467,9401fad4c497e1f9,203092
1580468,aacc960c9a228b5f,203092
1580469,d9e338c530dca106,203092


In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1580470 entries, 0 to 1580469
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           1580470 non-null  object
 1   landmark_id  1580470 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 110.0 MB
